# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 3 2022 10:17AM,249958,16,Emersa-8728383,"Emersa Waterbox, LLC",Released
1,Nov 3 2022 10:05AM,249952,15,VT80180164,"Virtus Pharmaceuticals, LLC",Executing
2,Nov 3 2022 10:05AM,249952,15,VT80180165,"Virtus Pharmaceuticals, LLC",Released
3,Nov 3 2022 10:05AM,249952,15,VT80180166,"Virtus Pharmaceuticals, LLC",Released
4,Nov 3 2022 9:48AM,249957,19,ACG-2102491758,ACG North America LLC,Executing
5,Nov 3 2022 9:40AM,249956,19,N17832,"NAPP Technologies, LLC",Released
6,Nov 3 2022 9:40AM,249950,15,30011906,"Alliance Pharma, Inc.",Released
7,Nov 3 2022 9:40AM,249950,15,30011907,"Alliance Pharma, Inc.",Released
8,Nov 3 2022 9:40AM,249950,15,30011908,"Alliance Pharma, Inc.",Released
9,Nov 3 2022 9:40AM,249950,15,30011909,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,249953,Released,4
27,249955,Released,1
28,249956,Released,1
29,249957,Executing,1
30,249958,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249953,11.0,4.0
249955,NaN,1.0
249956,NaN,1.0
249957,1.0,NaN
249958,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249881,0.0,1.0
249884,0.0,1.0
249885,0.0,1.0
249887,0.0,1.0
249888,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249881,0,1
249884,0,1
249885,0,1
249887,0,1
249888,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249881,0,1
1,249884,0,1
2,249885,0,1
3,249887,0,1
4,249888,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249881,,1
1,249884,,1
2,249885,,1
3,249887,,1
4,249888,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC"
1,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC"
4,Nov 3 2022 9:48AM,249957,19,ACG North America LLC
5,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC"
6,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc."
51,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc."
52,Nov 3 2022 9:31AM,249953,10,Bio-PRF
67,Nov 3 2022 9:30AM,249951,15,"Alliance Pharma, Inc."
69,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc."
70,Nov 3 2022 7:43AM,249938,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",,1
1,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",1,2
2,Nov 3 2022 9:48AM,249957,19,ACG North America LLC,1,
3,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",,1
4,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",12,33
5,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc.",,1
6,Nov 3 2022 9:31AM,249953,10,Bio-PRF,11,4
7,Nov 3 2022 9:30AM,249951,15,"Alliance Pharma, Inc.",,2
8,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc.",1,
9,Nov 3 2022 7:43AM,249938,19,"AdvaGen Pharma, Ltd",,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1,
1,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",2,1
2,Nov 3 2022 9:48AM,249957,19,ACG North America LLC,,1
3,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",1,
4,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",33,12
5,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc.",1,
6,Nov 3 2022 9:31AM,249953,10,Bio-PRF,4,11
7,Nov 3 2022 9:30AM,249951,15,"Alliance Pharma, Inc.",2,
8,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc.",,1
9,Nov 3 2022 7:43AM,249938,19,"AdvaGen Pharma, Ltd",4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1,
1,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",2,1
2,Nov 3 2022 9:48AM,249957,19,ACG North America LLC,,1
3,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",1,
4,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",33,12


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1.0,NaN
1,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",2.0,1.0
2,Nov 3 2022 9:48AM,249957,19,ACG North America LLC,NaN,1.0
3,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",1.0,NaN
4,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",33.0,12.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1.0,0.0
1,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",2.0,1.0
2,Nov 3 2022 9:48AM,249957,19,ACG North America LLC,0.0,1.0
3,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",1.0,0.0
4,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",33.0,12.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,249953,4.0,11.0
12,249945,0.0,1.0
15,999786,55.0,23.0
16,749730,3.0,0.0
19,1499660,22.0,21.0
21,2748829,11.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,249953,4.0,11.0
1,12,249945,0.0,1.0
2,15,999786,55.0,23.0
3,16,749730,3.0,0.0
4,19,1499660,22.0,21.0
5,21,2748829,11.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,4.0,11.0
1,12,0.0,1.0
2,15,55.0,23.0
3,16,3.0,0.0
4,19,22.0,21.0
5,21,11.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,4.0
1,12,Released,0.0
2,15,Released,55.0
3,16,Released,3.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Executing,11.0,1.0,23.0,0.0,21.0,0.0
Released,4.0,0.0,55.0,3.0,22.0,11.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Executing,11.0,1.0,23.0,0.0,21.0,0.0
1,Released,4.0,0.0,55.0,3.0,22.0,11.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Executing,11.0,1.0,23.0,0.0,21.0,0.0
1,Released,4.0,0.0,55.0,3.0,22.0,11.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()